### Prapengolahan

In [1]:
import pandas as pd
import numpy as np
import os
import olah_data as od
pd.options.display.max_columns = None

In [2]:
# DEFINE DATA MENTAH

# kriteria = 'call center'
# kriteria_call_center = 'car insurance'
# kriteria = 'twitter'
kriteria = 'email'

file_data_mentah = 'data mentah '+kriteria+'.xlsx'
file_bobot = "bobot/bobot perhitungan "+kriteria+".xlsx"

data_mentah = pd.read_excel(file_data_mentah)

# LOAD DATA BOBOT
daftar_bobot = pd.read_excel(file_bobot, sheet_name='indeks')
data_bobot = od.data_bobot(daftar_bobot)

In [3]:
# GANTI NAMA BULAN KE NOMOR BULAN
data_mentah.Bulan = od.ganti_bulan(data_mentah.Bulan,'nama bulan','nomor bulan')

# DROP NULL ROW
data_mentah = od.drop_null_row(data_mentah)

# DEFINE LIST BRAND
list_brand = data_mentah['Brand'].unique()
if kriteria == 'call center':
    brand_ivr, brand_nonivr = od.get_ivr_nonivr(list_brand, kriteria_call_center)

# TABEL CROSSTAB
crosstab_rata_transpose= od.crosstab_rata_transpose(data_mentah, list_brand)

### perhitungan indeks

In [4]:
# DEFINE LIST BULAN
list_bulan= pd.Series(data_mentah.Bulan.unique())
list_bulan = list(od.ganti_bulan(list_bulan,'nomor bulan','nama bulan'))


if kriteria=='call center':
    nilai_subaspek = od.nilai_sub_aspek(kriteria, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose,
                                       kriteria_call_center)
    nilai_aspek = od.nilai_aspek_with_sub(kriteria, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose, nilai_subaspek,
                                         kriteria_call_center)    
elif kriteria=='email':
    nilai_aspek = od.nilai_aspek_with_sub(kriteria, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose, None)
else:
    nilai_subaspek = od.nilai_sub_aspek(kriteria, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose)
    nilai_aspek = od.nilai_aspek_with_sub(kriteria, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose, nilai_subaspek)    
nilai_aspek_only = od.nilai_aspek_without_sub(kriteria, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose, nilai_aspek)
df_aspek_pivot = od.df_aspek_pivot(nilai_aspek, nilai_aspek_only, list_bulan)
tabel_summary_aspek = od.tabel_summary(df_aspek_pivot, list_bulan)


if kriteria=='call center':
    df_subkpi_pivot = od.df_subkpi_pivot(kriteria, daftar_bobot, list_bulan, list_brand, df_aspek_pivot,
                                        kriteria_call_center)
    df_kpi_pivot = od.df_kpi_pivot(kriteria, daftar_bobot, list_bulan, list_brand, df_subkpi_pivot,
                                  kriteria_call_center)
    df_dimensi_pivot = od.df_dimensi_pivot(kriteria, daftar_bobot, list_bulan, list_brand, df_kpi_pivot,
                                          kriteria_call_center)
    df_ccsei_pivot = od.df_ccsei_pivot(kriteria, daftar_bobot, list_bulan, list_brand, df_dimensi_pivot,
                                      kriteria_call_center)    
else:
    df_subkpi_pivot = od.df_subkpi_pivot(kriteria, daftar_bobot, list_bulan, list_brand, df_aspek_pivot)
    df_kpi_pivot = od.df_kpi_pivot(kriteria, daftar_bobot, list_bulan, list_brand, df_subkpi_pivot)
    df_dimensi_pivot = od.df_dimensi_pivot(kriteria, daftar_bobot, list_bulan, list_brand, df_kpi_pivot)
    df_ccsei_pivot = od.df_ccsei_pivot(kriteria, daftar_bobot, list_bulan, list_brand, df_dimensi_pivot)    
    
    
tabel_summary_subkpi = od.tabel_summary(df_subkpi_pivot, list_bulan)
tabel_summary_kpi = od.tabel_summary(df_kpi_pivot, list_bulan)
tabel_summary_dimensi = od.tabel_summary(df_dimensi_pivot, list_bulan)


# Load Data Bobot Engagement
bobot_engagement = pd.read_excel(file_bobot, sheet_name='engagement')
if kriteria=='call center':
    tabel_be_avg, df_eng_final_pivot = od.engagement(kriteria, bobot_engagement, df_aspek_pivot, list_bulan, list_brand, 
                                                     None, kriteria_call_center)
elif kriteria=='twitter':
    tabel_be_avg, df_eng_final_pivot = od.engagement(kriteria, bobot_engagement, df_aspek_pivot, list_bulan, list_brand)    
else:
    tabel_be_avg, df_eng_final_pivot = od.engagement(kriteria, bobot_engagement, df_aspek_pivot, list_bulan, list_brand, 
                                                     crosstab_rata_transpose)
    
    
# Load Data Bobot Impact perbaikan
bobot_perbaikan = pd.read_excel(file_bobot, sheet_name='perbaikan')
if kriteria=='call center':
    area_perbaikan = od.area_perbaikan(kriteria, data_mentah, bobot_perbaikan, list_bulan, list_brand,
                                      kriteria_call_center)
else:
    area_perbaikan = od.area_perbaikan(kriteria, data_mentah, bobot_perbaikan, list_bulan, list_brand)    
    
index_by_kpi, index_by_dimensi = od.tabel_indeks(tabel_summary_kpi, tabel_summary_dimensi, list_bulan)    

In [5]:
# df_ccsei_pivot

### Simpan Data ke Ms Excel

In [6]:
nama_output = "tabel output/tabel output "+kriteria+".xlsx"

list_simpan = [data_mentah, crosstab_rata_transpose, tabel_summary_aspek, tabel_summary_subkpi, tabel_summary_kpi,
              tabel_summary_dimensi, index_by_kpi, index_by_dimensi, df_ccsei_pivot, tabel_be_avg,
              df_eng_final_pivot, area_perbaikan]

list_nama_simpan = ['data mentah','crosstab rata-rata','aspek','subkpi','kpi','dimensi','index by kpi','index by dimensi','index akhir',
                   'engagement index aspek','engagement index final','aspek perbaikan']

writer = pd.ExcelWriter(nama_output)
for i in range(len(list_simpan)):
    if (list_nama_simpan[i]=='data mentah'):
        list_simpan[i].to_excel(writer,sheet_name=list_nama_simpan[i], index=False)
    else:
        list_simpan[i].to_excel(writer,sheet_name=list_nama_simpan[i])
writer.save() 